In [ ]:
import altair as alt
import pandas as pd
import datetime
import numpy as np

alt.data_transformers.disable_max_rows()

# QUESTION 1

In [ ]:
df = pd.read_csv('BoonsongLekagulwaterways.csv')

In [ ]:
df.head()

In [ ]:
df.shape

## Finding 1.1

In [ ]:
df.head()

In [ ]:
# Calculate the interquartile range (IQR) and define the upper and lower bounds for outliers
Q1 = df['value'].quantile(0.25)
Q3 = df['value'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_outliers = df[(df['value'] < lower_bound) | (df['value'] > upper_bound)]
num_outliers = len(df) - len(df_outliers)
num_outliers = len(df_outliers)
df_outliers = df[(df['value'] < lower_bound) | (df['value'] > upper_bound)]
num_outliers = len(df_outliers)

In [ ]:
# Create a list of measures for the dropdown menu
measures = list(df['measure'].unique())

# Define the dropdown menu
dropdown = alt.binding_select(options=measures)
select_measure = alt.selection_single(fields=['measure'], bind=dropdown, name='Select', init={'measure': 'Magnesium'})

# Plot the number of outliers using Altair
chart1 = alt.Chart(pd.DataFrame({'Outliers': [num_outliers]})).mark_bar().encode(
    x=alt.X('Outliers:Q', title='Number of Outliers'),
    y=alt.Y(),
    tooltip=[alt.Tooltip('Outliers', title='Number of Outliers')]
).properties(
    title='Number of Outliers in Data',
)

# Create the scatter plot of the outliers with the dropdown menu
chart2 = alt.Chart(df_outliers).mark_circle().encode(
    x='location',
    y=alt.Y('value:Q', title='Value'),
    color=alt.condition(select_measure, 'measure', alt.value('lightgray')),
    tooltip=['id', 'value', 'location', 'measure']
).add_selection(
    select_measure
).transform_filter(
    select_measure
).properties(
    title='Scatter Plot of Outliers',
    height=300,
    width=200
)

# Combine the two charts and display
(chart1).display()
(chart2).display()


In [ ]:
outlier_counts = df_outliers.groupby('measure')['value'].count().reset_index(name='count')
chart = alt.Chart(outlier_counts).mark_bar().encode(
    x='measure:N',
    y='count:Q',
    tooltip=[alt.Tooltip('count:Q', title='Number of Outliers')]
).properties(
    title='Number of Outliers by Chemical',
    width=400,
    height=300
)

chart.display()

In [ ]:
df1 = df.copy()

In [ ]:
df2 = df1.copy()

In [ ]:
df2['sample date']=pd.to_datetime(df2['sample date'])
df2['year']=df2['sample date'].dt.year
df2['month']=df2['sample date'].dt.month
df2['day'] = df2['sample date'].dt.day
df2.head()

## Finding 1.2

In [ ]:
chart_width = 1000
chart_height = 300
chart_title = 'Observations Count per Location from 1998 to 2016'

base = alt.Chart(df2).transform_aggregate(
    measure='count()',
    groupby=['location', 'year']
).encode(
    x=alt.X('year:O', scale=alt.Scale(paddingInner=0.05)),
    y=alt.Y('location:N', scale=alt.Scale(paddingInner=0.05))
)

heatmap = base.mark_rect().encode(
    color=alt.Color('measure:Q',
                    scale=alt.Scale(scheme='greens'),
                    legend=alt.Legend(direction='horizontal')),
    tooltip=['year', 'location', 'measure']
)

text = base.mark_text(align='left').encode(
    text='measure:Q',
    color=alt.condition(
        alt.datum.measure > 500,
        alt.value('black'),
        alt.value('red')
    )
)

combined_chart = (heatmap + text).properties(
    width=chart_width,
    height=chart_height,
    title=chart_title
)

combined_chart

# QUESTION 2

In [ ]:
df2.head(0)

In [ ]:
df3 = df2.copy()

## Finding 2.1

In [ ]:
# DROPDOWN FEATURE CREATION
default_measure = 'Petroleum hydrocarbons'
input_features_dropdown = alt.binding_select(options=df3['measure'].unique().tolist(), name='measure')
selection = alt.selection_single(fields=['measure'], bind=input_features_dropdown, init={'measure': default_measure})

# Create the trend chart with a dropdown feature
chart = alt.Chart(df3).mark_line().encode(
    x=alt.X('year:O', title="Year"),
    y=alt.Y('mean(value):Q', title="Mean of Values"),
    color=alt.Color('location:N', scale=alt.Scale(scheme="rainbow"), legend=alt.Legend(title="Location")),
    tooltip=['year', 'location', 'mean(value)'],
).properties(
    title="Contamination trend across all measures in different locations",
    width=700,
    height=500,
).add_selection(selection).transform_filter(selection)

chart.display()


In [ ]:
df['sample date'] = pd.to_datetime(df['sample date'])
df_monthly = df.groupby(['location', 'measure', pd.Grouper(key='sample date', freq='M')])['value'].mean().reset_index()
df_monthly['change_rate'] = df_monthly['value'].pct_change()

# dropping rows with the value of inf in the change_rate column
df_monthly = df_monthly.replace([np.inf, -np.inf], np.nan)
df_monthly = df_monthly.dropna()

df_monthly[df_monthly['change_rate'] > 10]['measure'].unique()

## Finding 2.2

In [ ]:
# Define default value for dropdown feature
default_feature = 'Water temperature'

# DROPDOWN FEATURE CREATION
input_features_dropdown = alt.binding_select(options=df3['measure'].unique().tolist(), name='measure')
selection = alt.selection_single(fields=['measure'], bind=input_features_dropdown, name='Select', init={'measure': default_feature})

# Filter the data based on the selected measure
df_measure = df3[df3['measure'] == selection['measure']]

# Creating the trend chart
contamination_trend_line = alt.Chart(df_measure).mark_line(point=alt.OverlayMarkDef(color="red")).encode(
    x='year',
    y='mean(value):Q',
    tooltip=['location', 'year', 'mean(value):Q']
).properties(width=750, height=100, title=alt.TitleParams(text= 'Trend Over Years'))

annotation = alt.Chart(df_measure).mark_text(
    align='center', baseline='line-bottom', fontSize=11, dx=1
).encode(
    x=alt.X('year'),
    y=alt.Y('mean(value):Q'),
    text=alt.Text('mean(value):O', format='.1f')
)

chart = alt.layer(contamination_trend_line, annotation, data=df_measure).facet(row='location')

chart = chart.add_selection(selection).transform_filter(selection)

chart.display()
